# NLP homework (Naive Bayes, Logistic Regression, Fasttext)

## loading data 

In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# download necessary stuff
import os
import spacy
try:
    spacy_en = spacy.load("en_core_web_sm")
except:
    os.system('python -m spacy download en_core_web_sm')
    spacy_en = spacy.load("en_core_web_sm")
    
import nltk
for package_name in ['wordnet', 'omw-1.4', 'stopwords', 'punkt']:
    nltk.download(package_name)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#!pip install datasets

In [6]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("emotion")
class_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger', 
    4: 'fear', 
    5: 'surprise'
}

Using custom data configuration default
Reusing dataset emotion (C:\Users\user\.cache\huggingface\datasets\emotion\default\0.0.0\348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
train_df = pd.DataFrame({'POST': dataset['train']['text'], 'LABEL': dataset['train']['label']})
val_df = pd.DataFrame({'POST': dataset['validation']['text'], 'LABEL': dataset['validation']['label']})
test_df = pd.DataFrame({'POST': dataset['test']['text'], 'LABEL': dataset['test']['label']})

In [64]:
train_df


,POST,LABEL
0,im feeling rather rotten so im not very ambiti...,0
1,im updating my blog because i feel shitty,0
2,i never make her separate from me because i do...,3
3,i left with my bouquet of red and yellow tulip...,2
4,i was feeling a little vain when i did this one,3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3



# Perform EDA

Perform data exploration and make some plots.

In [67]:
def lemmatize_spacy(text):
    text = spacy_en(text)
    #print(text)
    lemmas = [token.lemma_ for token in text]
    return " ".join(lemmas)

In [74]:
remove_stopwords(lemmatize_spacy(train_df["POST"][0]),stops_spacy )


'I m feel rather rotten so I m not very ambitious right now'

In [ ]:
###############################################################
###########################################
train_df['POST'] = train_df['POST'].map(lemmatize_spacy) 
###why doesnt it  work ?????????

In [76]:
for i in range(len(train_df)):
    train_df["POST"][i] = lemmatize_spacy(train_df["POST"][i])

C:\Users\user\AppData\Local\Temp/ipykernel_9092/3691644766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["POST"][i] = lemmatize_spacy(train_df["POST"][i])


In [78]:
train_df

,POST,LABEL
0,I m feel rather rotten so I m not very ambitio...,0
1,I m update my blog because I feel shitty,0
2,I never make she separate from I because I don...,3
3,I leave with my bouquet of red and yellow tuli...,2
4,I be feel a little vain when I do this one,3
...,...,...
15995,I just have a very brief time in the beanbag a...,0
15996,I be now turn and I feel pathetic that I be st...,0
15997,I feel strong and good overall,1
15998,I feel like this be such a rude comment and I ...,3


In [91]:
stops_spacy = sorted(spacy.lang.en.stop_words.STOP_WORDS)
from nltk.corpus import stopwords
stops = sorted(stopwords.words("english"))
def remove_stopwords(text, stopwords):
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

In [92]:
for i in range(len(train_df["POST"])):
    train_df["POST"][i] = remove_stopwords(str(train_df["POST"][i]), stops_spacy)


C:\Users\user\AppData\Local\Temp/ipykernel_9092/3863513540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["POST"][i] = remove_stopwords(str(train_df["POST"][i]), stops_spacy)


In [93]:
train_df

,POST,LABEL
0,I m feel rotten I m ambitious right,0
1,I m update blog I feel shitty,0
2,I separate I I don t want feel like I m ashamed,3
3,I leave bouquet red yellow tulip arm feel slig...,2
4,I feel little vain I,3
...,...,...
15995,I brief time beanbag I anna I feel like I beat,0
15996,I turn I feel pathetic I wait table sub teachi...,0
15997,I feel strong good overall,1
15998,I feel like rude comment I m glad t,3


In [56]:
type(train_df["POST"][0])

spacy.tokens.doc.Doc

# Preprocess 

Based on the EDA, decide which preprocessing steps you should use.

,POST,LABEL
0,"(i, m, feeling, rather, rotten, so, i, m, not,...",0
1,"(i, m, updating, my, blog, because, i, feel, s...",0
2,"(i, never, make, her, separate, from, me, beca...",3
3,"(i, left, with, my, bouquet, of, red, and, yel...",2
4,"(i, was, feeling, a, little, vain, when, i, di...",3
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,1
15998,i feel like this was such a rude comment and i...,3


# Bayes 

# Train

Train at least one Naive Bayes classifier, one Logistic Regression model and one Fasttext model. For Naive Bayes and Logistic Regression models you can try different vectorization methods. For all the models you can try different tokenization methods. Check the scores both on train and validation data.

In [113]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [126]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [127]:
nb = Pipeline([('countVec', CountVectorizer(lowercase=False, token_pattern='\w+', min_df=3)),
               ('clf', MultinomialNB()),])
nb.fit(train_df["POST"], train_df["LABEL"])


Pipeline(steps=[('countVec',
                 CountVectorizer(lowercase=False, min_df=3,
                                 token_pattern='\\w+')),
                ('clf', MultinomialNB())])

In [128]:
y_pred = nb.predict(val_df["POST"])
train_score =  round(accuracy_score(y_pred, val_df["LABEL"]), 3)
print(train_score)

0.8


In [130]:
nb = Pipeline([('countVec', CountVectorizer(lowercase=False, token_pattern='\w+', ngram_range=(1, 3), min_df=3)),
               ('clf', MultinomialNB()),])
nb.fit(train_df["POST"], train_df["LABEL"])
y_pred = nb.predict(val_df["POST"])
train_score =  round(accuracy_score(y_pred, val_df["LABEL"]), 3)
print(train_score)

0.803


In [131]:
nb = Pipeline([('tfidf', TfidfVectorizer(lowercase=False, token_pattern='\w+', ngram_range=(1, 2), 
                                         min_df=3)),
               ('clf', MultinomialNB()),])
nb.fit(train_df["POST"], train_df["LABEL"])
y_pred = nb.predict(val_df["POST"])
train_score =  round(accuracy_score(y_pred, val_df["LABEL"]), 3)
print(train_score)

0.71


# logistic

In [121]:
from sklearn.linear_model import LogisticRegression

In [132]:
logreg = Pipeline([('countVec', CountVectorizer(lowercase=False, token_pattern='\w+', min_df=3)),
                   ('clf', LogisticRegression(random_state=42, solver='liblinear')),])
logreg.fit(train_df["POST"], train_df["LABEL"])
y_pred = logreg.predict(val_df["POST"])
train_score =  round(accuracy_score(y_pred, val_df["LABEL"]), 3)
print(train_score)

0.841


In [133]:
logreg = Pipeline([('countVec', CountVectorizer(lowercase=False, token_pattern='\w+', ngram_range=(1, 2), 
                                                min_df=3)),
                   ('clf', LogisticRegression(random_state=42, solver='liblinear')),])

logreg.fit(train_df["POST"], train_df["LABEL"])
y_pred = logreg.predict(val_df["POST"])
train_score =  round(accuracy_score(y_pred, val_df["LABEL"]), 3)
print(train_score)

0.832


In [259]:
logreg = Pipeline([('tfidf', TfidfVectorizer(lowercase=False, token_pattern='\w+', ngram_range=(1, 2), 
                                             min_df=3)),
                   ('clf', LogisticRegression(random_state=42, solver='liblinear')),])

logreg.fit(train_df["POST"], train_df["LABEL"])
y_pred = logreg.predict(val_df["POST"])

train_score =  round(accuracy_score(y_pred, val_df["LABEL"]), 3)
print(train_score)

0.82


# Fasttext

In [240]:
from fasttext import train_supervised


In [257]:
from fasttext import train_supervised

def to_fasttext_format(data: list, labels: list, save_path: str=None):
    ft_data = []
    for d, l in zip(data, labels):
        ft_data.append("__label__{} {}".format(l, d))
    if save_path:
        np.savetxt(save_path, ft_data, fmt='%s')
    else:
        return ft_data
    
def train_fasttext(X_train, y_train, wordNgrams=1, minCount=1, ft_train_path="./untitled.txt", **kwargs):
    to_fasttext_format(X_train, y_train, save_path=ft_train_path)
    ft_model = train_supervised(ft_train_path, wordNgrams=wordNgrams, minCount=minCount, epoch=10, loss="softmax",  **kwargs)
    print(ft_model)
    train_preds = [i[0].split('_')[-1] for i in ft_model.predict(list(X_train))[0]]
    result = [int(item) for item in train_preds]
    train_score = accuracy_score(result, y_train, 3)
    print("train_score = ",train_score)
    
    return ft_model, train_score

In [258]:
ft_model, train_score = train_fasttext(train_df["POST"], train_df["LABEL"])
train_score


train_score =  0.8855625


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass normalize=3 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


0.8855625

In [254]:
preds = [i[0].split('_')[-1] for i in ft_model.predict(list(val_df["POST"]))[0]]
val_preds = [int(item) for item in preds]
val_score =accuracy_score(val_preds, val_df["LABEL"], 3)
print(f'val accuracy {val_score}')

val accuracy 0.8245


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass normalize=3 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [264]:
ft_model, train_score = train_fasttext(train_df["POST"], train_df["LABEL"], wordNgrams=2)
preds = [i[0].split('_')[-1] for i in ft_model.predict(list(val_df["POST"]))[0]]
val_preds = [int(item) for item in preds]

val_score = accuracy_score(val_preds,val_df["LABEL"] , 3)

print(f'val accuracy {val_score}')

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass normalize=3 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


train_score =  0.9499375
val accuracy 0.8155


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass normalize=3 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [265]:
ft_model, train_score = train_fasttext(train_df["POST"], train_df["LABEL"], minn=4, maxn=5)

preds = [i[0].split('_')[-1] for i in ft_model.predict(list(val_df["POST"]))[0]]
val_preds = [int(item) for item in preds]

val_score = accuracy_score(val_preds,val_df["LABEL"] , 3)

print(f'val accuracy {val_score}')

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass normalize=3 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


train_score =  0.8175
val accuracy 0.795


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass normalize=3 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [ ]:
#!pip install fasttext

# Check results on Test Data

From each algorithm you used, choose the best model based on the validation score, and check its score on test data.

# Best one is logistic regression

In [267]:
logreg = Pipeline([('countVec', CountVectorizer(lowercase=False, token_pattern='\w+', min_df=3)),
                   ('clf', LogisticRegression(random_state=42, solver='liblinear')),])
logreg.fit(train_df["POST"], train_df["LABEL"])
y_pred = logreg.predict(val_df["POST"])
val_score =  round(accuracy_score(y_pred, val_df["LABEL"]), 3)

y_pred = logreg.predict(test_df["POST"])
test_score =  round(accuracy_score(y_pred,test_df["LABEL"]), 3)

print("val_score =", val_score, "\ntest_score = ",test_score)

val_score = 0.841 
test_score =  0.802


In [ ]:
!nautilus .